ADD NOTEBOOK INTRO TEXT HERE

### Notebook Setup: Import Packages and Authenticate

In [86]:
import json
import os
import pathlib
import time

import requests
import planet
from planet import Session, DataClient, OrdersClient

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

#### Testing Hosted Data API Connection

Let's start by testing our API connection to ensure that everything is working properly.

In [47]:
hd_url = 'https://api.planet.com/hosted-data/v1/folders'

In [49]:
# Here, we want to see a 200 Response.
response = requests.get(hd_url, auth=session.auth)
response

<Response [200]>

We can also look at our response to see any existing Hosted Data API folders you may have access to.

In [50]:
response.json()

{'folders': [{'id': '76OgKpDgx3hHM4R66B8j',
   'name': 'test_folder',
   'description': '',
   'created': '2023-01-17T23:16:35.327273Z',
   'updated': '2023-01-17T23:16:35.327273Z',
   'active_count': 0,
   '_owner': True,
   '_permissions': [{'allow': True, 'action': 'can-read'},
    {'allow': True, 'action': 'can-write'}],
   '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/76OgKpDgx3hHM4R66B8j',
    'contents': 'https://api.planet.com/hosted-data/v1/folders/76OgKpDgx3hHM4R66B8j/contents'}},
  {'id': 'rbQe4cwTwnuCsYGoq5Vk',
   'name': 'simple order',
   'description': '',
   'created': '2023-01-17T23:06:41.112929Z',
   'updated': '2023-01-17T23:06:41.112929Z',
   'active_count': 0,
   '_owner': True,
   '_permissions': [{'allow': True, 'action': 'can-read'},
    {'allow': True, 'action': 'can-write'}],
   '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/rbQe4cwTwnuCsYGoq5Vk',
    'contents': 'https://api.planet.com/hosted-data/v1/folders/rbQe4cwTwnuC

### Use the Hosted Data API to Create & Delete Folders

#### Create a Hosted Data Folder

In [64]:
# set content type to json
headers = {'content-type': 'application/json'}

In [80]:
# include the name of the folder you would like to create in the request
request = {"name":"test_folder"}

In [66]:
response = requests.post(hd_url, data=json.dumps(request), auth=session.auth, headers=headers)
response

<Response [201]>

A 201 Response means that our folder has been created!

We can see our new folder `test_folder` by looking at the json response:

In [67]:
response.json()

{'id': 'KxhbEZUIsSNtBSZc4G8b',
 'name': 'test_folder1',
 'description': '',
 'created': '2023-01-17T23:39:30.579933Z',
 'updated': '2023-01-17T23:39:30.579933Z',
 'active_count': 0,
 '_owner': True,
 '_permissions': [{'allow': True, 'action': 'can-read'},
  {'allow': True, 'action': 'can-write'}],
 '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/KxhbEZUIsSNtBSZc4G8b',
  'contents': 'https://api.planet.com/hosted-data/v1/folders/KxhbEZUIsSNtBSZc4G8b/contents'}}

#### Delete a Hosted Data Folder

In [81]:
# Get the folder URL using the ID of the folder we just created
folder_url = hd_url + '/' + response.json()['id']

In [82]:
# Send a delete request to the API
response = requests.delete(folder_url, auth=session.auth)
response

<Response [204]>

A 204 Response means that our request has succeeded, and there's no more info that needs to be returned to the response body.

We can confirm this by listing out our folders again. Notice that our `test_folder` is gone!

In [83]:
response = requests.get(hd_url, auth=session.auth)
response.json()

{'folders': [{'id': '3gIdabfXpEGyJKB710Jh',
   'name': 'Cleveland waterfront',
   'description': '',
   'created': '2021-07-21T14:59:35.467Z',
   'updated': '2022-09-20T19:39:44.597Z',
   'active_count': 5,
   '_owner': False,
   '_permissions': [{'allow': True, 'action': 'can-read'}],
   '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/3gIdabfXpEGyJKB710Jh',
    'contents': 'https://api.planet.com/hosted-data/v1/folders/3gIdabfXpEGyJKB710Jh/contents'}},
  {'id': 'afMB5gjjxsmXnCG2of5I',
   'name': 'Port of Long Beach Monitoring',
   'description': '',
   'created': '2021-03-17T21:28:44.364Z',
   'updated': '2021-03-17T22:52:34.28Z',
   'active_count': 4,
   '_owner': False,
   '_permissions': [{'allow': True, 'action': 'can-read'}],
   '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/afMB5gjjxsmXnCG2of5I',
    'contents': 'https://api.planet.com/hosted-data/v1/folders/afMB5gjjxsmXnCG2of5I/contents'}},
  {'id': 'g7GoGYrJEbSWgL3nLcKM',
   'name': 'Magesh

### Order Images to a Hosted Data Folder Using the Orders API

Let's start by creating a folder once more since we deleted our original folder above.

In [84]:
# Create our `test_folder` once again
response = requests.post(hd_url, data=json.dumps(request), auth=session.auth, headers=headers)
response.json()

{'id': 'a0pbNXEEKh2etbFOKovd',
 'name': 'test_folder',
 'description': '',
 'created': '2023-01-17T23:51:42.731723Z',
 'updated': '2023-01-17T23:51:42.731723Z',
 'active_count': 0,
 '_owner': True,
 '_permissions': [{'allow': True, 'action': 'can-read'},
  {'allow': True, 'action': 'can-write'}],
 '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/a0pbNXEEKh2etbFOKovd',
  'contents': 'https://api.planet.com/hosted-data/v1/folders/a0pbNXEEKh2etbFOKovd/contents'}}

In [94]:
folder_id = response.json()['id']

Now, we can order images to this folder using the Orders API. There are a few steps involved:
1) Identify data to be ordered  
2) Build the order request  
3) Create and deliver the order to our hosted data folder

#### Identify Data To Be Ordered

In [85]:
# The area of interest (AOI) defined as a polygon
iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

# The item IDs we wish to order
iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

#### Build the Order Request

We don't yet have SDK suport for Hosted Data order requests so we'll need to build this json manually for now.

In this example, we are looking at some `item_ids` that correspond to `PSScene` images for a location in Iowa. For more information and examples on how to identify the item IDs corresponding to your needs, please see the [Data API documentation on our Developer Center](https://developers.planet.com/docs/apis/data/) and our Data API [jupyter notebooks](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials).

In [163]:
# Note that we're using the `folder_id` we just created.

iowa_order ={
'name': 'iowa_order',
 'products': [{'item_ids': ['20200925_161029_69_2223', '20200925_161027_48_2223'],
               'item_type': 'PSScene',
               'product_bundle': 'analytic_sr_udm2'}],
    'delivery': {
        'planet_folders': {
            'folder_id': folder_id}}}

In [164]:
# view our order request
iowa_order

{'name': 'iowa_order',
 'products': [{'item_ids': ['20200925_161029_69_2223',
    '20200925_161027_48_2223'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_sr_udm2'}],
 'delivery': {'planet_folders': {'folder_id': 'a0pbNXEEKh2etbFOKovd'}}}

#### Create and Deliver the Order

First, we will create a function that will allow us to to create the order and wait until it is completed. Then, we will run this function using the Planet Orders Client.

In [172]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

In [166]:
async with planet.Session() as ps:
    # The Orders API client
    client = planet.OrdersClient(ps)
    # Create the order and deliver it to our Hosted Data Folder
    order_details = await create_and_deliver_order(iowa_order, client)

04:31 - order 1796ec73-1151-471b-a91e-99b4ff7cbccb - state: success


In [167]:
# Let's look at the details of the order that we just placed.
order_details

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/1796ec73-1151-471b-a91e-99b4ff7cbccb',
  'results': [{'delivery': 'success',
    'expires_at': '0001-01-01T00:00:00.000Z',
    'name': 'org-1/user:345186/folder-a0pbNXEEKh2etbFOKovd/1796ec73-1151-471b-a91e-99b4ff7cbccb/PSScene/20200925_161027_48_2223_3B_udm2.tif'},
   {'delivery': 'success',
    'expires_at': '0001-01-01T00:00:00.000Z',
    'name': 'org-1/user:345186/folder-a0pbNXEEKh2etbFOKovd/1796ec73-1151-471b-a91e-99b4ff7cbccb/PSScene/20200925_161027_48_2223_3B_AnalyticMS_metadata.xml'},
   {'delivery': 'success',
    'expires_at': '0001-01-01T00:00:00.000Z',
    'name': 'org-1/user:345186/folder-a0pbNXEEKh2etbFOKovd/1796ec73-1151-471b-a91e-99b4ff7cbccb/PSScene/20200925_161027_48_2223_3B_AnalyticMS_SR.tif'},
   {'delivery': 'success',
    'expires_at': '0001-01-01T00:00:00.000Z',
    'name': 'org-1/user:345186/folder-a0pbNXEEKh2etbFOKovd/1796ec73-1151-471b-a91e-99b4ff7cbccb/PSScene/20200925_161027_48_2223_metadata.j

Note that now, when we look at at the first item in our Hosted Data URL response, we see that our test folder has been populated, and the `updated` field reflects when the new items were added.

In [170]:
response = requests.get(hd_url, auth=session.auth)
response.json()['folders'][0]

{'id': 'a0pbNXEEKh2etbFOKovd',
 'name': 'test_folder',
 'description': '',
 'created': '2023-01-17T23:51:42.731723Z',
 'updated': '2023-01-18T23:21:09.078909Z',
 'active_count': 2,
 '_owner': True,
 '_permissions': [{'allow': True, 'action': 'can-read'},
  {'allow': True, 'action': 'can-write'}],
 '_links': {'self': 'https://api.planet.com/hosted-data/v1/folders/a0pbNXEEKh2etbFOKovd',
  'contents': 'https://api.planet.com/hosted-data/v1/folders/a0pbNXEEKh2etbFOKovd/contents'}}

### Listing Images in Hosted Data Folders as STAC

TBD

### Stream a Hosted Data COG

TBD

### Stream an XYZ Tile Service for Images in Hosted Data Folders 

TBD